In [1]:
import argparse
import pandas as pd
from src.Utils.Dictionaries import team_index_current
from src.Utils.tools import get_json_data, to_data_frame, get_todays_games_json, create_todays_games
from colorama import Fore, Style, init, deinit

from pycaret.classification import *

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
todays_games_url = 'https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2021/scores/00_todays_scores.json'
data_url = 'https://stats.nba.com/stats/leaguedashteamstats?' \
           'Conference=&DateFrom=&DateTo=&Division=&GameScope=&' \
           'GameSegment=&LastNGames=0&LeagueID=00&Location=&' \
           'MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&' \
           'PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&' \
           'PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&' \
           'Season=2021-22&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&' \
           'StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='

In [4]:
def expected_value(Pwin, odds):
    """
    In betting, the expected value (EV) is the measure of what a bettor 
    can expect to win or lose per bet placed on the same odds time and time again. 
    Positive expected value (+EV) implies profit over time, 
    while a negative value (-EV) implies a loss over time.
    
    """
    Ploss = 1 - Pwin
    Mwin = payout(odds)
    return round((Pwin * Mwin) - (Ploss * 100), 2)


def payout(odds):
    if odds > 0:
        return odds
    else:
        return (100 / (-1 * odds)) * 100

In [5]:
def createTodaysGames(games, team_stats_df):
    
    team_stats_dicts = team_stats_df.to_dict(orient='records')
    
    todays_game_dicts=[]
    
    for game in games:
        home_team=game['home_team']
        away_team=game['away_team']
        date=game['date']
        ou=input(f'[HOME] {home_team} vs [AWAY] {away_team} Over and Under : ')
        spread=input(f'[HOME] {home_team} vs [AWAY] {away_team} Spread : ')
        ml_home=input(f'[HOME] {home_team} Money Line : ')
        ml_away=input(f'[AWAY] {away_team} Money Line : ')
        
        #Team info dict
        team_info_dict={
            'date': game['date'],
            'home': home_team,
            'away': away_team,
            'ou': ou,
            'spread': spread,
            'ml_home': ml_home,
            'ml_away': ml_away            
            
        }

        #Get stats
        home_team_stats_list=[dic for dic in team_stats_dicts if dic['TEAM_NAME']==home_team]
        away_team_stats_list=[dic for dic in team_stats_dicts if dic['TEAM_NAME']==away_team]

        #Filter for stats
        home_team_stats_dict={k+'_HOME': v for k, v in home_team_stats_list[0].items() if k not in ['TEAM_ID', 'CFID', 'CFPARAMS']}
        away_team_stats_dict={k+'_AWAY': v for k, v in away_team_stats_list[0].items() if k not in ['TEAM_ID', 'CFID', 'CFPARAMS']}

        #Combine stats and odds
        team_result_dict = {**team_info_dict, **home_team_stats_dict, **away_team_stats_dict}

        todays_game_dicts.append(team_result_dict)

    return todays_game_dicts

In [6]:
def get_expected_values(pred_df):
    
    pred_dict = pred_df.to_dict(orient='records')
    
    for game in pred_dict:

        Pwin_home = round(game['Score_W'],4)
        Pwin_away = round(game['Score_L'],4)

        odds_home = int(game['ml_home'])
        odds_away = int(game['ml_away'])

        home_team_ev = expected_value(Pwin_home, odds_home)
        away_team_ev = expected_value(Pwin_away, odds_away)

        game['home_team_ml_expected_value'] = home_team_ev
        game['away_team_ml_expected_value'] = away_team_ev
        
    return pred_dict

In [7]:
def predict(todays_game_dicts):
    
    pred_df = pd.DataFrame(todays_game_dicts)
    pred_df.columns = pred_df.columns.str.lower()

    #load models
    win_loss_model = load_model('win_loss_acc_72')
    ou_model = load_model('ou_cover_acc_56')
    
    #make predictions
    win_loss_prediction_df = predict_model(win_loss_model, data = pred_df, raw_score = True)
    ou_prediction_df = predict_model(ou_model, data = pred_df, raw_score = True)
    
    #Get ML expect values
    win_loss_results = get_expected_values(win_loss_prediction_df)
    ou_results = ou_prediction_df.to_dict(orient='records')
    
    return win_loss_results, ou_results

In [34]:
def color_selection(home_value,away_value,method='greater_less'):
    if method=='greater_less':
        
        if home_value>away_value:
            home_color=Fore.GREEN
            away_color=Fore.RED
        else:
            home_color=Fore.RED
            away_color=Fore.GREEN
            
        return home_color,away_color
    
    elif method=='positive_negative':
        
        if home_value>0:
            home_color=Fore.GREEN
        else: 
            home_color=Fore.RED
        if away_value>0:
            away_color=Fore.GREEN
        else: 
            away_color=Fore.RED
        return home_color,away_color

In [42]:
def print_results(win_loss_results, ou_results):
    
    for game in win_loss_results:
        
        date = game['date']
        home_team = game['home']
        away_team = game['away']
        home_team_ml_odds = game['ml_home']
        away_team_ml_odds = game['ml_away']
        home_team_w_prob = round(game['Score_W']*100,4)
        away_team_w_prob = round(game['Score_L']*100,4)
        home_w_color,away_w_color=color_selection(home_team_w_prob,
                                                  away_team_w_prob, 
                                                  method='greater_less')
        
        home_team_ml_expected_value = game['home_team_ml_expected_value']
        away_team_ml_expected_value = game['away_team_ml_expected_value']
        home_ev_color,away_ev_color=color_selection(home_team_ml_expected_value,
                                                  away_team_ml_expected_value,
                                                  method='positive_negative')
        
        ou_dict = [dic for dic in ou_results if dic['home']==home_team and dic['away']==away_team][0]
        over = round(ou_dict['Score_Over']*100,4)
        under = round(ou_dict['Score_Under']*100,4)
        ou_color,under_color=color_selection(over,
                                              under,
                                              method='greater_less')

        print(f'------ {date} [HOME] {Fore.BLUE }{home_team}{Style.RESET_ALL} vs. [AWAY] {Fore.MAGENTA}{away_team}{Style.RESET_ALL} ----')            
        print(f'[ML ODDS]           {home_team} ( {home_team_ml_odds} ) | {away_team} ( {away_team_ml_odds} )')
        print(f'[ML WIN CONFIDENCE] {home_team} ( {home_w_color}{home_team_w_prob}{Style.RESET_ALL} % ) | {away_team} ( {away_w_color}{away_team_w_prob}{Style.RESET_ALL} % )')
        print(f'[ML EXPECTED VALUE] {home_team} ( {home_ev_color}${home_team_ml_expected_value}{Style.RESET_ALL}  ) | {away_team} ( {away_ev_color}${away_team_ml_expected_value}{Style.RESET_ALL}  )')
        print(f'[OVER & UNDER]       OVER ( {ou_color}{over}{Style.RESET_ALL} % ) | UNDER ( {under_color}{under}{Style.RESET_ALL} % ) ')
        print('')

In [9]:
data = get_todays_games_json(todays_games_url)

In [10]:
games = create_todays_games(data)

In [11]:
team_stats = get_json_data(data_url)

In [12]:
team_stats_df = to_data_frame(team_stats)

In [13]:
todays_game_dicts = createTodaysGames(games, team_stats_df)

[HOME] Atlanta Hawks vs [AWAY] Milwaukee Bucks Over and Under : 232.5
[HOME] Atlanta Hawks vs [AWAY] Milwaukee Bucks Spread : 5.5
[HOME] Atlanta Hawks Money Line : 170
[AWAY] Milwaukee Bucks Money Line : -200
[HOME] Memphis Grizzlies vs [AWAY] Chicago Bulls Over and Under : 225.5
[HOME] Memphis Grizzlies vs [AWAY] Chicago Bulls Spread : 8.5
[HOME] Memphis Grizzlies Money Line : -350
[AWAY] Chicago Bulls Money Line : 275
[HOME] Orlando Magic vs [AWAY] Portland Trail Blazers Over and Under : 218.5
[HOME] Orlando Magic vs [AWAY] Portland Trail Blazers Spread : 2.5
[HOME] Orlando Magic Money Line : 115
[AWAY] Portland Trail Blazers Money Line : -140
[HOME] Los Angeles Lakers vs [AWAY] Utah Jazz Over and Under : 230.5
[HOME] Los Angeles Lakers vs [AWAY] Utah Jazz Spread : 4.5
[HOME] Los Angeles Lakers Money Line : 155
[AWAY] Utah Jazz Money Line : -190
[HOME] Miami Heat vs [AWAY] Toronto Raptors Over and Under : 207.5
[HOME] Miami Heat vs [AWAY] Toronto Raptors Spread : 3.5
[HOME] Miami Hea

In [14]:
win_loss_results, ou_results = predict(todays_game_dicts)

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [43]:
print_results(win_loss_results, ou_results)

------ 2022-01-17 [HOME] Atlanta Hawks vs. [AWAY] Milwaukee Bucks ----
[ML ODDS]           Atlanta Hawks ( 170 ) | Milwaukee Bucks ( -200 )
[ML WIN CONFIDENCE] Atlanta Hawks ( 32.28 % ) | Milwaukee Bucks ( 67.72 % )
[ML EXPECTED VALUE] Atlanta Hawks ( $-12.84  ) | Milwaukee Bucks ( $1.58  )
[OVER & UNDER]       OVER ( 28.56 % ) | UNDER ( 70.21 % ) 

------ 2022-01-17 [HOME] Memphis Grizzlies vs. [AWAY] Chicago Bulls ----
[ML ODDS]           Memphis Grizzlies ( -350 ) | Chicago Bulls ( 275 )
[ML WIN CONFIDENCE] Memphis Grizzlies ( 70.83 % ) | Chicago Bulls ( 29.17 % )
[ML EXPECTED VALUE] Memphis Grizzlies ( $-8.93  ) | Chicago Bulls ( $9.39  )
[OVER & UNDER]       OVER ( 25.74 % ) | UNDER ( 71.09 % ) 

------ 2022-01-17 [HOME] Orlando Magic vs. [AWAY] Portland Trail Blazers ----
[ML ODDS]           Orlando Magic ( 115 ) | Portland Trail Blazers ( -140 )
[ML WIN CONFIDENCE] Orlando Magic ( 18.19 % ) | Portland Trail Blazers ( 81.81 % )
[ML EXPECTED VALUE] Orlando Magic ( $-60.89  ) | Por